# Size 열 전처리

In [ ]:
df_info['Size'].replace('Varies with device', np.nan, inplace=True)

In [ ]:
df_info['Size']=df_info['Size'].astype(str)

def k_to_m(x):
    if 'k' in x:
        return float(x.replace('k', ''))/1000
    else: return str(x)

df_info['Size']=df_info['Size'].apply(k_to_m)

df_info['Size']=df_info['Size'].apply(lambda x: x.replace('M',''))

df_info['Size']=df_info['Size'].apply(lambda x: float(x))

In [ ]:
df_info['Size'].fillna(df_info.groupby('Category')['Size'].transform('mean'),inplace = True)
df_info['Size']=np.round(df_info['Size'],decimals=1)

# 전체 리뷰 Word Cloud

In [ ]:
all_rev=df_review['Translated_Review'].tolist()

all_rev=' '.join(all_rev)

import re
all_text=re.sub("[^a-zA-Z]"," ", all_rev) #문자를 제외한 나머지 요소 공백으로 바꿈
all_text=all_text.lower() #소문자화


import nltk
from nltk.corpus import stopwords
all_text=nltk.word_tokenize(all_text) #Tokenize
all_text_filtered=[word for word in all_text if word not in stopwords.words('english')] #Stopwords 제거

import nltk as nlp
lemma=nlp.WordNetLemmatizer() #Lemmatization (apps -> app 이런 식으로)
all_text_lemma=[lemma.lemmatize(i) for i in all_text_filtered]
all_text=[" ".join(all_text_lemma)]


FONT_PATH='/Users/sejeongan/Library/Fonts/Product Sans Regular.ttf'
mask=np.array(Image.open('/Users/sejeongan/Downloads/732208.jpg'))

plt.subplots(figsize=(10,10))
wordcloud=WordCloud(
    background_color="white",
    width=300,
    height=300,
    font_path=FONT_PATH,
    color_func=ImageColorGenerator(mask),
    mask=mask).generate_from_text(" ".join(all_text))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

# Sentiment별 Word Cloud

In [ ]:
sent_dct = {}
for sent in df_review.Sentiment.unique().tolist():

    print(sent)
    sent_reviews =df_review[df_review['Sentiment'] == sent]
    
    all_rev=df_review['Translated_Review'].tolist()

    all_rev=' '.join(all_rev)

    import re
    all_text=re.sub("[^a-zA-Z]"," ", all_rev) 
    all_text=all_text.lower() 

    import nltk
    from nltk.corpus import stopwords
    all_text=nltk.word_tokenize(all_text)
    all_text_filtered=[word for word in all_text if word not in stopwords.words('english')]

    import nltk as nlp
    lemma=nlp.WordNetLemmatizer()
    all_text_lemma=[lemma.lemmatize(i) for i in all_text_filtered] 
    all_text=[" ".join(all_text_lemma)]


#Word Cloud 생성
FONT_PATH='/Users/sejeongan/Library/Fonts/Product Sans Regular.ttf'
mask=np.array(Image.open('/Users/sejeongan/Downloads/732208.jpg'))

plt.subplots(figsize=(10,10))
wordcloud=WordCloud(
    background_color="white",
    width=300,
    height=300,
    font_path=FONT_PATH,
    color_func=ImageColorGenerator(mask),
    mask=mask).generate_from_text(" ".join(all_text))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

# Category별 Word Cloud

In [ ]:
review2=df_review.groupby('App').mean()

review_apps=review2.index.tolist()
app_with_reviews=[]
for name in review_apps:
    if name in df_info['App'].unique().tolist():
        app_with_reviews.append(name)
        
drop_lst=[]
for x in range(0,len(df_info)):
    if df_info.App[x] not in app_with_reviews or df_info.App[x] in df_info.App.iloc[:x].values:
        drop_lst.append(x)
appdata2=df_info.drop(index=drop_lst).reset_index(drop=True)


drop_lst2=[]
for x in range(0,len(review2)):
    if review2.index[x] not in app_with_reviews:
        drop_lst2.append(review2.index[x])
review3=review2.drop(index=drop_lst2)
        
data=pd.merge(appdata2,review3, left_on='App', right_on='App')
        
        
df_review_only=df_review.drop(columns=['Sentiment', 'Sentiment_Polarity', 'Sentiment_Subjectivity'])

df_review_only2=df_review_only.drop(df_review_only[~df_review_only['App'].isin(app_with_reviews)].index)


review_only3=df_review_only2.groupby(['App'], as_index=False).agg({'Translated_Review':' '.join})

lst=[]
for k in review_only3.App:
    for index in range(0,len(data)):
        if data.App[index] == k:
            lst.append(data.Category[index])
s1 = pd.Series(lst)

review_only3['Category']=s1
review_only3=review_only3[['App', 'Category', 'Translated_Review']]

In [ ]:
cat_dct = {}
for cat in df_info.Category.unique().tolist():

    print(cat)
    category_reviews=review_only3[review_only3['Category']==cat]
    cat_rev=category_reviews['Translated_Review'].tolist()
    cat_rev=' '.join(cat_rev)
    for k in ['love','app','great','work','time','good','really','like','thank']:
        cat_rev=cat_rev.replace(k,'')

    cat_text=re.sub("[^a-zA-Z]"," ", cat_rev) #문자를 제외한 나머지 요소 공백으로 바꿈
    cat_text=cat_text.lower() #소문자화

    cat_text=nltk.word_tokenize(cat_text)
    cat_text_filtered=[word for word in cat_text if word not in stopwords.words('english')]
    
    lemma=nltk.WordNetLemmatizer()
    cat_text_lemma=[lemma.lemmatize(i) for i in cat_text_filtered]
    cat_text=[" ".join(cat_text_lemma)]
    
    
    FONT_PATH='/Users/sejeongan/Library/Fonts/Product Sans Regular.ttf'
    mask=np.array(Image.open('/Users/sejeongan/Downloads/732208.jpg'))
    
    plt.subplots(figsize=(10,10))
    wordcloud=WordCloud(
        background_color="white",
        width=300,
        height=300,
        font_path=FONT_PATH,
        color_func=ImageColorGenerator(mask),
        mask=mask).generate_from_text(" ".join(cat_text))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

# 1. 카테고리 명 2. 상위권 리뷰 Word Cloud 3. 중위권 리뷰 Word Cloud

In [ ]:
#리뷰 존재하는 상위권(upper outlier) 앱 추출
def out_with_reviews(category):
    a_lst=[]
    install_lst=[]
    app_lst=[]
    index_lst=[]
    for x in range(0,len(df_info)):
        if df_info.Category[x] == category:
            app_lst.append(df_info.App[x])
            install_lst.append(df_info.Installs[x])
    a = np.array(install_lst)
    q1,q3= np.percentile(a,[25,75])
    IQR = q3 - q1
    lower = q1 - (IQR*1.5)
    upper = q3 + (IQR*1.5)
    
    for index,value in enumerate(a):
        if (value >= upper):
            if app_lst[index] in (app_with_reviews):
                a_lst.append(app_lst[index])

    for k in range(0,len(review_only3)):
        if review_only3.App[k] in a_lst:
            index_lst.append(True)
        else:
            index_lst.append(False)
    
    return(np.array(index_lst))
                
    return a_lst 

In [ ]:
#리뷰 존재하는 outlier 아닌 앱 추출
def out_removed_reviews(category):
    a_lst=[]
    install_lst=[]
    app_lst=[]
    index_lst=[]
    for x in range(0,len(df_info)):
        if df_info.Category[x] == category:
            app_lst.append(df_info.App[x])
            install_lst.append(df_info.Installs[x])
    a = np.array(install_lst)
    q1,q3= np.percentile(a,[25,75])
    IQR = q3 - q1
    lower = q1 - (IQR*1.5)
    upper = q3 + (IQR*1.5)
    
    for index,value in enumerate(a):
        if (lower<= value <= upper):
            if app_lst[index] in (app_with_reviews):
                a_lst.append(app_lst[index])
    
    for k in range(0,len(review_only3)):
        if review_only3.App[k] in a_lst:
            index_lst.append(True)
        else:
            index_lst.append(False)
    
    return(np.array(index_lst))

    return (a_lst)

In [ ]:
FONT_PATH='/Users/sejeongan/Library/Fonts/Product Sans Regular.ttf'
mask=np.array(Image.open('/Users/sejeongan/Downloads/732208.jpg'))

cat_dct = {}
for cat in df_info.Category.unique().tolist():

    print(cat)
    category_reviews=review_only3[out_with_reviews(cat)]
    cat_rev=category_reviews['Translated_Review'].tolist()
    cat_rev=' '.join(cat_rev)
    for k in ['love','app','great','work','time','good','really','like','thank']:
        cat_rev=cat_rev.replace(k,'')

    cat_text=re.sub("[^a-zA-Z]"," ", cat_rev) #문자를 제외한 나머지 요소 공백으로 바꿈
    cat_text=cat_text.lower() #소문자화

    cat_text=nltk.word_tokenize(cat_text)
    cat_text_filtered=[word for word in cat_text if word not in stopwords.words('english')]
    
    lemma=nltk.WordNetLemmatizer()
    cat_text_lemma=[lemma.lemmatize(i) for i in cat_text_filtered]
    cat_text=[" ".join(cat_text_lemma)]
    
    try:
        plt.subplots(figsize=(10,10))
        wordcloud=WordCloud(
            background_color="white",
            width=300,
            height=300,
            font_path=FONT_PATH,
            color_func=ImageColorGenerator(mask),
            mask=mask).generate_from_text(" ".join(cat_text))
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.show()
    except:
        print('결과 없음')
        pass
    
    dog_reviews=review_only3[out_removed_reviews(cat)]
    dog_rev=dog_reviews['Translated_Review'].tolist()
    dog_rev=' '.join(dog_rev)
    for k in ['love','app','great','work','time','good','really','like','thank']:
        dog_rev=dog_rev.replace(k,'')

    dog_text=re.sub("[^a-zA-Z]"," ", dog_rev)
    dog_text=dog_text.lower()

    dog_text=nltk.word_tokenize(dog_text)
    dog_text_filtered=[word for word in dog_text if word not in stopwords.words('english')]
    
    lemma=nltk.WordNetLemmatizer()
    dog_text_lemma=[lemma.lemmatize(i) for i in dog_text_filtered]
    dog_text=[" ".join(dog_text_lemma)]
    
    try:
        plt.subplots(figsize=(10,10))
        wordcloud=WordCloud(
            background_color="white",
            width=300,
            height=300,
            font_path=FONT_PATH,
            color_func=ImageColorGenerator(mask),
            mask=mask).generate_from_text(" ".join(dog_text))
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.show()
    except:
        print('결과 없음')
        pass